# Mastercard Misuse of Integrity Fee

In [261]:
import pyodbc #CONNECT VIA PYODBC IMPORT THE PYODBC LIBRARY
import pandas as pd #CONNECT TO THE PANDAS LIBRARY
import numpy as np #IMPORT NUMPY
import matplotlib.pyplot as plt
print pyodbc.dataSources() #DISPLAY THE LISTED DATA SOURCES ON MACHINE, IN MY CASE I USE NETEZZASQL


{'ECL_CODA11': 'Oracle in instantclient_11_2', 'NZSQL': 'NetezzaSQL', 'IST_CODA11': 'Oracle in instantclient_11_2', 'BO_Lookup': 'Oracle in instantclient_11_2', 'Excel Files': 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'dBASE Files': 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'MS Access Database': 'Microsoft Access Driver (*.mdb, *.accdb)'}


In [ ]:
import_curr ='G:\Pricing\IC Opt project\pi_implementation\currency conversion\currency_data_11042017.csv'

## Import the Currency Conversion Information from the Currency Monitor

In [262]:
curr=pd.read_csv(import_curr)
curr=curr[['CURRENCYCODE','USD_AMOUNT']]

## Run the Neteeza SQL Statement Identifying those with a Valid Misuse of Integrity Status

In [14]:
mc_mu_data=pd.read_sql(
'''
SELECT extract(month from CAST(pa.ATTEMPTCREATEDDATE AS DATE))AS STATUSMONTH,m.CLIENT_NAME,pa.MERCHANTID,m.CONTRACT_ID,m.CLIENT_ID,pa.ORDERID
		,pa.REQUESTAUTHORIZEDAMOUNT,pa.BILLINGCURRENCYCODE AS CURRENCYCODE,pa.CURRENT_PAYMENTSTATUSID_UPDATED
		,pa.CREDITDEBITINDICATOR,cs.CARD_SCHEME_FULL,sp.SERVICEPROVIDERGROUPNAME,sp.GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC
FROM P_MMDB.MDW.BI_OPR_PAYMENTATTEMPT1 AS pa

--JOIN TOGETHER THE MERCHANT INFORMATION
LEFT JOIN DATA.MB_MERCHANTS AS m
ON pa.MERCHANTID=m.CONTRACT_ID

--JOIN TOGETHER THE SERVICE PROVIDER INFORMATION
--WITH THE PRIMARY DATASET, CAN NOW FILTER FOR JUST THE EUROPE
LEFT JOIN P_MMDB.MDW.DIM_SERVICEPROVIDER AS sp
ON pa.SERVICEPROVIDERID=sp.SERVICEPROVIDERID

--PULL IN THE CARD_SCHME DATA AND COMBINE WITH THE PRIMARY DATASET
LEFT JOIN P_MMDB.MDW.DIM_CARD_SCHEME AS cs
ON pa.PAYMENTPRODUCTID =cs.PAYMENTPRODUCTID

--LEFT JOIN DATA.CC_PROCESSING_COSTLINK_VIEW pcv
--ON pa.MERCHANTID =pcv.CONTRACT_ID AND pa.ORDERID =pcv.ORDERID

--PULL DATA DOWN JUST FOR THE US
WHERE ATTEMPTCREATEDDATE BETWEEN '2017-01-01' AND '2017-12-31'
AND (sp.GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC NOT LIKE '%EEA%' AND sp.GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC NOT LIKE '%Europe%')
AND (CURRENT_PAYMENTSTATUSID_UPDATED=525 OR CURRENT_PAYMENTSTATUSID_UPDATED=600) 
AND(CARD_SCHEME_FULL='MasterCard' )
--GROUP THE DATA TO ALLOW FOR SUMMATIONS
GROUP BY MERCHANTID,m.CONTRACT_ID,CLIENT_NAME,CURRENT_PAYMENTSTATUSID_UPDATED,CREDITDEBITINDICATOR,CURRENCYCODE,SERVICEPROVIDERGROUPNAME,GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC
                ,CARD_SCHEME_FULL,STATUSMONTH,CLIENT_ID,pa.ORDERID,pa.REQUESTAUTHORIZEDAMOUNT
'''
,conn)
             

In [263]:
cc_data=pd.read_sql(
'''
SELECT cc.PAYMENT_PROCESSOR,cc.PRODUCT_CODE,cc.ICPLUS_TYPE,cc.CONTRACT_ID,cc.INVOICE_CURRENCY
FROM DATA.CC_PROCESSING_DATA_PT AS cc
WHERE TRANSACTION_DATE BETWEEN '2017-01-01' AND '2017-12-31'
AND cc.TRANSACTION_TYPE='TRANSACTION'
--AND cc.CONTRACT_ID=4235
--AND cc.PAYMENT_PROCESSOR='FIRSTDATA'
--AND cc.PRODUCT_CODE=100453
AND cc.INVOICE_CURRENCY IS NOT NULL
GROUP BY cc.PAYMENT_PROCESSOR,cc.PRODUCT_CODE,cc.ICPLUS_TYPE,cc.CONTRACT_ID,cc.INVOICE_CURRENCY
'''
,conn)

In [264]:
mc_mu_working_data=mc_mu_data
print mc_mu_working_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431652 entries, 0 to 431651
Data columns (total 16 columns):
STATUSMONTH                              431652 non-null int64
CLIENT_NAME                              431652 non-null object
MERCHANTID                               431652 non-null int64
CONTRACT_ID                              431652 non-null float64
CLIENT_ID                                431652 non-null float64
ORDERID                                  431652 non-null int64
REQUESTAUTHORIZEDAMOUNT                  431652 non-null float64
CURRENCYCODE                             431652 non-null object
CURRENT_PAYMENTSTATUSID_UPDATED          431652 non-null int64
CREDITDEBITINDICATOR                     431652 non-null object
CARD_SCHEME_FULL                         431652 non-null object
SERVICEPROVIDERGROUPNAME                 431652 non-null object
GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC    431652 non-null object
0.25%_P/T                                431652 non-null f

In [265]:
cc_data_working_data=cc_data
print cc_data_working_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14959 entries, 0 to 14958
Data columns (total 5 columns):
PAYMENT_PROCESSOR    14955 non-null object
PRODUCT_CODE         14959 non-null float64
ICPLUS_TYPE          14959 non-null object
CONTRACT_ID          14959 non-null float64
INVOICE_CURRENCY     14959 non-null object
dtypes: float64(2), object(3)
memory usage: 584.4+ KB
None


In [266]:
mc_mu_working_data['CLIENT_NAME']=mc_mu_working_data['CLIENT_NAME'].str.strip()
mc_mu_working_data['CURRENCYCODE']=mc_mu_working_data['CURRENCYCODE'].str.strip()
mc_mu_working_data['CREDITDEBITINDICATOR']=mc_mu_working_data['CREDITDEBITINDICATOR'].str.strip()
mc_mu_working_data['SERVICEPROVIDERGROUPNAME']=mc_mu_working_data['SERVICEPROVIDERGROUPNAME'].str.strip()
mc_mu_working_data['GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC']=mc_mu_working_data['GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC'].str.strip()

In [267]:
cc_data_working_data['PAYMENT_PROCESSOR']=cc_data_working_data['PAYMENT_PROCESSOR'].str.strip()
cc_data_working_data['ICPLUS_TYPE']=cc_data_working_data['ICPLUS_TYPE'].str.strip()

In [268]:
cc_data_working_data.rename(columns={'PAYMENT_PROCESSOR': 'SERVICEPROVIDERGROUPNAME'}, inplace=True)

In [269]:
cc_data_working_data=cc_data_working_data[(cc_data_working_data['SERVICEPROVIDERGROUPNAME']=='WELLS FARGO')
                                          |(cc_data_working_data['SERVICEPROVIDERGROUPNAME']=='Vantiv')
                                          |(cc_data_working_data['SERVICEPROVIDERGROUPNAME']=='Merchant Solutions')
                                          |(cc_data_working_data['SERVICEPROVIDERGROUPNAME']=='Elavon (USA)')
                                          |(cc_data_working_data['SERVICEPROVIDERGROUPNAME']=='FIRSTDATA')
                                          |(cc_data_working_data['SERVICEPROVIDERGROUPNAME']=='COMMONWEALTH')]

In [270]:
import1='G:\\Pricing\\IC Opt project\\pi_implementation\\misuse of authorization fee\\reference_tables\\product_ref_tbl.csv'
prd_ref_tbl=pd.read_csv(import1)
prd_ref_tbl=prd_ref_tbl[['CREDITDEBITINDICATOR','CURRENCYCODE','PRODUCT_CODE',]]
print prd_ref_tbl.head()

  CREDITDEBITINDICATOR CURRENCYCODE  PRODUCT_CODE
0                    C          BRL        100454
1                    C          CAD        100454
2                    C          CLP        100454
3                    C          COP        100454
4                    C          CRC        100454


In [271]:
mc_mu_working_data['0.25%_P/T']=mc_mu_working_data['REQUESTAUTHORIZEDAMOUNT']*.0025
mc_mu_working_data['0.04_P/T']=1*0.04
mc_mu_working_data['PROCESS_INTEGRITY_FEE']=mc_mu_working_data[['0.25%_P/T','0.04_P/T']].max(axis=1)

In [272]:
mc_mu_working_data=mc_mu_working_data[(mc_mu_working_data['SERVICEPROVIDERGROUPNAME']=='Merchant Solutions')
                                      |(mc_mu_working_data['SERVICEPROVIDERGROUPNAME']=='FirstData')
                                      |(mc_mu_working_data['SERVICEPROVIDERGROUPNAME']=='CBA')
                                      |(mc_mu_working_data['SERVICEPROVIDERGROUPNAME']=='Litle')
                                      |(mc_mu_working_data['SERVICEPROVIDERGROUPNAME']=='Wells Fargo')
                                      |(mc_mu_working_data['SERVICEPROVIDERGROUPNAME']=='Elavon')]

In [273]:
mc_mu_working_data['SERVICEPROVIDERGROUPNAME'][mc_mu_working_data['SERVICEPROVIDERGROUPNAME'].str.contains('Litle')]='Vantiv'
mc_mu_working_data['SERVICEPROVIDERGROUPNAME'][mc_mu_working_data['SERVICEPROVIDERGROUPNAME'].str.contains('Wells Fargo')]='WELLS FARGO'
mc_mu_working_data['SERVICEPROVIDERGROUPNAME'][mc_mu_working_data['SERVICEPROVIDERGROUPNAME'].str.contains('FirstData')]='FIRSTDATA'
mc_mu_working_data['SERVICEPROVIDERGROUPNAME'][mc_mu_working_data['SERVICEPROVIDERGROUPNAME'].str.contains('Elavon')]='Elavon (USA)'
mc_mu_working_data['SERVICEPROVIDERGROUPNAME'][mc_mu_working_data['SERVICEPROVIDERGROUPNAME'].str.contains('CBA')]='COMMONWEALTH'
mc_mu_working_data.info()

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  a

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411617 entries, 0 to 431651
Data columns (total 16 columns):
STATUSMONTH                              411617 non-null int64
CLIENT_NAME                              411617 non-null object
MERCHANTID                               411617 non-null int64
CONTRACT_ID                              411617 non-null float64
CLIENT_ID                                411617 non-null float64
ORDERID                                  411617 non-null int64
REQUESTAUTHORIZEDAMOUNT                  411617 non-null float64
CURRENCYCODE                             411617 non-null object
CURRENT_PAYMENTSTATUSID_UPDATED          411617 non-null int64
CREDITDEBITINDICATOR                     411617 non-null object
CARD_SCHEME_FULL                         411617 non-null object
SERVICEPROVIDERGROUPNAME                 411617 non-null object
GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC    411617 non-null object
0.25%_P/T                                411617 non-null f

In [274]:
mc_mu_working_data=mc_mu_working_data[['STATUSMONTH','CLIENT_NAME','CONTRACT_ID','SERVICEPROVIDERGROUPNAME','GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC','ORDERID','CREDITDEBITINDICATOR','CURRENCYCODE','REQUESTAUTHORIZEDAMOUNT','PROCESS_INTEGRITY_FEE']]

In [275]:
mc_mu_working_data=pd.merge(mc_mu_working_data,prd_ref_tbl,on=['CURRENCYCODE','CREDITDEBITINDICATOR'],how='left')

In [276]:
mc_mu_working_data_final=pd.merge(mc_mu_working_data,cc_data_working_data,on=['CONTRACT_ID','PRODUCT_CODE','SERVICEPROVIDERGROUPNAME'],how='left')

In [277]:
mc_mu_working_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411637 entries, 0 to 411636
Data columns (total 13 columns):
STATUSMONTH                              411637 non-null int64
CLIENT_NAME                              411637 non-null object
CONTRACT_ID                              411637 non-null float64
SERVICEPROVIDERGROUPNAME                 411637 non-null object
GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC    411637 non-null object
ORDERID                                  411637 non-null int64
CREDITDEBITINDICATOR                     411637 non-null object
CURRENCYCODE                             411637 non-null object
REQUESTAUTHORIZEDAMOUNT                  411637 non-null float64
PROCESS_INTEGRITY_FEE                    411637 non-null float64
PRODUCT_CODE                             411637 non-null int64
ICPLUS_TYPE                              411552 non-null object
INVOICE_CURRENCY                         411552 non-null object
dtypes: float64(3), int64(3), object(7)
memory usage: 44.

In [278]:
mc_mu_working_data_final=pd.merge(mc_mu_working_data_final,curr,on=['CURRENCYCODE'],how='left')
mc_mu_working_data_final.rename(columns={'USD_AMOUNT': 'USD_CONVERSION_FACTOR'}, inplace=True)

In [279]:
mc_mu_working_data_final['USD_ADJUSTED_PROCESS_INTEGRITY_FEE']=0

In [280]:
mc_mu_working_data_final['USD_ADJUSTED_PROCESS_INTEGRITY_FEE'][(mc_mu_working_data_final['PROCESS_INTEGRITY_FEE']!=0.04)&(mc_mu_working_data_final['CURRENCYCODE']!='USD')]=mc_mu_working_data_final['PROCESS_INTEGRITY_FEE']*mc_mu_working_data_final['USD_CONVERSION_FACTOR']
mc_mu_working_data_final['USD_ADJUSTED_PROCESS_INTEGRITY_FEE'][(mc_mu_working_data_final['USD_ADJUSTED_PROCESS_INTEGRITY_FEE']==0)]=mc_mu_working_data_final['PROCESS_INTEGRITY_FEE']

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [286]:
curr_invoice=curr
curr_invoice.rename(columns={'CURRENCYCODE': 'INVOICE_CURRENCY'}, inplace=True)
curr_invoice

mc_mu_working_data_final=pd.merge(mc_mu_working_data_final,curr_invoice,on=['INVOICE_CURRENCY'],how='left')
mc_mu_working_data_final['INVOICE_CONVERSION_FACTOR']=mc_mu_working_data_final['USD_AMOUNT']

mc_mu_working_data_final['CURRENCY_ADJUSTED_INVOICE_AMOUNT']=mc_mu_working_data_final['USD_ADJUSTED_PROCESS_INTEGRITY_FEE']/mc_mu_working_data_final['INVOICE_CONVERSION_FACTOR']

In [319]:
eur=curr[1:2].iloc[0]['USD_AMOUNT']#Establish the European Conversion Amount
mc_mu_working_data_final['EUR_TOTAL']=mc_mu_working_data_final['USD_ADJUSTED_PROCESS_INTEGRITY_FEE']/eur

In [320]:
mc_mu_working_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411637 entries, 0 to 411636
Data columns (total 18 columns):
STATUSMONTH                              411637 non-null int64
CLIENT_NAME                              411637 non-null object
CONTRACT_ID                              411637 non-null float64
SERVICEPROVIDERGROUPNAME                 411637 non-null object
PRODUCT_CODE                             411637 non-null int64
GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC    411637 non-null object
ORDERID                                  411637 non-null int64
CREDITDEBITINDICATOR                     411637 non-null object
CURRENCYCODE                             411637 non-null object
INVOICE_CURRENCY                         411552 non-null object
REQUESTAUTHORIZEDAMOUNT                  411637 non-null float64
PROCESS_INTEGRITY_FEE                    411637 non-null float64
USD_CONVERSION_FACTOR                    411637 non-null float64
USD_ADJUSTED_PROCESS_INTEGRITY_FEE       411637 non-null

In [321]:
mc_mu_working_data_final=mc_mu_working_data_final[['STATUSMONTH'
                          ,'CLIENT_NAME'
                          ,'CONTRACT_ID'
                          ,'SERVICEPROVIDERGROUPNAME'
                          ,'PRODUCT_CODE'
                          ,'GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC'
                          ,'ORDERID'
                          ,'CREDITDEBITINDICATOR'
                          ,'CURRENCYCODE'
                          ,'INVOICE_CURRENCY'                         
                          ,'REQUESTAUTHORIZEDAMOUNT'
                          ,'PROCESS_INTEGRITY_FEE'
                          ,'USD_CONVERSION_FACTOR'                                                   
                          ,'USD_ADJUSTED_PROCESS_INTEGRITY_FEE'
                          ,'INVOICE_CONVERSION_FACTOR'
                          ,'CURRENCY_ADJUSTED_INVOICE_AMOUNT'
                          ,'EUR_TOTAL'                         
                          ,'ICPLUS_TYPE']]

In [322]:
mc_mu_working_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411637 entries, 0 to 411636
Data columns (total 18 columns):
STATUSMONTH                              411637 non-null int64
CLIENT_NAME                              411637 non-null object
CONTRACT_ID                              411637 non-null float64
SERVICEPROVIDERGROUPNAME                 411637 non-null object
PRODUCT_CODE                             411637 non-null int64
GEOGRAPHICAL_AREA_LOCAL_ACQUIRER_DESC    411637 non-null object
ORDERID                                  411637 non-null int64
CREDITDEBITINDICATOR                     411637 non-null object
CURRENCYCODE                             411637 non-null object
INVOICE_CURRENCY                         411552 non-null object
REQUESTAUTHORIZEDAMOUNT                  411637 non-null float64
PROCESS_INTEGRITY_FEE                    411637 non-null float64
USD_CONVERSION_FACTOR                    411637 non-null float64
USD_ADJUSTED_PROCESS_INTEGRITY_FEE       411637 non-null

In [324]:
mc_mu_working_data_final.to_csv('d:\\djohnson\\Desktop\\mc_integ_fee.csv')